In [ ]:
!pip install --upgrade pip
!pip install mecab-python3

In [ ]:
import MeCab

tagger = MeCab.Tagger('-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd')
input = 'MeCabは 京都大学情報学研究科−日本電信電話株式会社コミュニケーション科学基礎研究所 共同研究ユニットプロジェクトを通じて開発されたオープンソース 形態素解析エンジンです。'
result = tagger.parse(input)
print(result)

In [ ]:
import sagemaker

In [ ]:
local_path = './'
sagemaker.s3.S3Downloader.download(
    s3_uri='s3://[your bucket name]/sample_text.json',
    local_path=local_path
)

In [ ]:
import json

words = []

with open('sample_text.json') as ndjson_dict:
    for line in ndjson_dict:
        data = json.loads(line)
        print(data['text'])
        result = tagger.parse(data['text'])
        print(result)
        nouns = [line for line in result.splitlines() if "名詞" in line.split()[-1]]

        for str in nouns:
            #print(str.split()[0])
            words.append(str.split()[0])

In [ ]:
print(words)

In [ ]:
import pandas as pd
sr = pd.Series(words)
sr.value_counts()

In [ ]:
!pip install japanize-matplotlib

import japanize_matplotlib

In [ ]:
sr.value_counts().plot.bar()

In [ ]:
# 単語のリストをデータフレームに変換する
import pandas as pd
words_df = pd.DataFrame({'word':words})

In [ ]:
words_df

In [ ]:
!pip install awswrangler

In [ ]:
import awswrangler as wr
import pandas as pd
# SageMakerのノートブックインスタンスのIAMに、AWSGlueConsoleFullAccessポリシー必要
# CREATE DATABASE  sample_sagemaker_db;でデータベースは事前に作っておく
wr.s3.to_parquet(
    df=words_df,
    path="s3://[your bucket name]/dataset/",
    dataset=True,
    database="sample_sagemaker_db",
    table="words_table"
)

In [ ]:
df = wr.athena.read_sql_query("SELECT word, count(*) AS cnt FROM words_table GROUP BY word", database="sample_sagemaker_db")

In [ ]:
df.sort_values('cnt', ascending=False)

In [ ]:
import boto3
import json

# Comprehend constant
REGION = 'us-west-2'
# Function for detecting sentiment
def detect_sentiment(text, language_code):
    comprehend = boto3.client('comprehend', region_name=REGION)
    response = comprehend.detect_sentiment(Text=text, LanguageCode=language_code)
    return response

In [ ]:
# # SageMakerのノートブックインスタンスのIAMに、ComprehendFullAccess
results = []
with open('sample_text.json') as ndjson_dict:
    for line in ndjson_dict:
        data = json.loads(line)
        print(data['text'])
        result = detect_sentiment(data['text'], 'ja')
        print("Starting detecting sentiment")
        print(json.dumps(result, sort_keys=True, indent=4))
        results.append({"text": data['text'], "sentiment": result["Sentiment"]})
        print("End of detecting sentiment\n")

In [ ]:
results

In [ ]:
results_df = pd.DataFrame(results)
results_df

In [ ]:
sentiment_group = results_df.groupby("sentiment") 
sentiment_group.size()